In [2]:
!pip install contractions

     |████████████████████████████████| 312 kB 820 kB/s eta 0:00:01
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp37-cp37m-linux_x86_64.whl size=99073 sha256=e267dda6b2381f5b962367e074e2ca1d7217e6611bcb6dec84b269d8d2fe83ba
  Stored in directory: /root/.cache/pip/wheels/9b/6b/f7/62dc8caf183b125107209c014e78c340a0b4b7b392c23c2db4
Successfully built pyahocorasick
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# 1. Importación de las librerias.

In [11]:
# Libreria equivalente a numpy optimizada para CUDA
import cupy as cp
import cupyx

# Libreria para el procesamiento de lenguaje natural NLP
import spacy

# Libreria para dividir un dataset en train y test
from sklearn.model_selection import train_test_split

# Permite vectorizar un text corpus (transforma texto a una secuencia de enteros)
from tensorflow.keras.preprocessing.text import Tokenizer

# Transforma una lista de secuencia de enteros a una lista normalizada.
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Permite trabajar con contracciones de palabras, por ejemplo Don't, We're, etc.
import contractions

# Permite normalizar cadenas de texto eliminando caracteres como tildes.
import unidecode

# Libreria para utilizar el algoritmo Naive Bayes a traves de CUDA.
from cuml.naive_bayes import MultinomialNB

# Libreria para trabajar con Regex
import re

# Libreria para el manejo de matrices y manejo de datos avanzados.
import pandas as pd

# Libreria enforcada en el manejo de matrices.
import numpy as np

# Libreria para medir el tiempo.
import time

# 2. Lectura de los 3 datasets de diferentes fuentes.

In [4]:
# Permite acceder a funciones del Sistema Operativo.
import os

# Leemos cada uno de los datasets con los que vamos a trabajar.
dataframe_yelp = pd.read_csv('/kaggle/input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt', delimiter='\t', names=['text', 'sentimiento'])
dataframe_amazon = pd.read_csv('/kaggle/input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt', delimiter='\t', names=['text', 'sentimiento'])
dataframe_imdb = pd.read_csv('/kaggle/input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt', delimiter='\t', names=['text', 'sentimiento'])


# Concatenamos los 3 datasets.
data = pd.concat([dataframe_yelp, dataframe_amazon, dataframe_imdb])


## 2.1 Imprimimos la cabecera de los 3 datasets previamente concatenados.

In [5]:
data.head()

,text,sentimiento
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


# 3. Splitting de Train y Test

In [6]:
# Se establece los pandas.Series del train y test, en la cual 70% del dataset va para el train y 30% del dataset se usa en el test.

x_train, x_test, y_train, y_test = train_test_split(data['text'], data['sentimiento'], test_size = 0.30, random_state=42)
value_test = x_test[0:20]
expected_test = y_test[0:20]

# 4. Creacion de las funciones para el preprocesamiento de datos.

In [7]:
"""
    Permite eliminar caracteres especiales que no sean letras y numeros.
"""
def quitar_caracteres_especiales(data):
    pattern = r'[^a-zA-z0-9\s]'
    for i in range(len(data)):
        data[i] = re.sub(pattern, '', data[i])
    return data
    

"""
    Quita los acentos de palabras, por ejemplo caffé pasaria a ser cafe.
"""
def quitar_acentos(data):
    for i in range(len(data)):
        data[i] = unidecode.unidecode(data[i])
    return data

"""
    Permite expandir las contracciones de palabras como Don't a Do not o We're a we are.
"""
def expandir_contracciones(data):
    for i in range(len(data)):
        data[i] = contractions.fix(data[i])
    return data


"""
    Lematiza las palabras, por ejemplo de Jumping a Jump, o Playing a Play, o Bought a Buy.
"""
def lematizar_palabras(data):
    nlp = spacy.load('en_core_web_lg')
    for i in range(len(data)):
        doc = nlp(data[i])
        tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in doc]
        data[i] = ' '.join(tokens)
    return data

"""
    Esta funcion se encarga de llamar a las funcionea antes definidas para procesar al dataset entero.
"""

def preprocesar(data):
    data = data.to_numpy()
    
    data = quitar_caracteres_especiales(data)
    data = quitar_acentos(data)
    data = expandir_contracciones(data)
    data = lematizar_palabras(data)
    
    data = pd.Series(data)
    
    # Realiza el proceso de la tokenizacion de cadenas de texto.
    tokenizer = Tokenizer(num_words=250, lower= 1, oov_token="<OOV>")
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    
    # Permite llenar una secuencia de enteros con 0 para igualar su tamano.
    padded_data = pad_sequences(sequences, padding='post', maxlen=350)
    return padded_data

# 5. Preprocesamiento de datos aplicado al Train y Test

In [8]:
# Preprocesamos nuestros pandas.Series, tanto en train como test.
x_train = preprocesar(x_train)
x_test = preprocesar(x_test)

# 6. Entrenamiento de nuestro clasificador Naive Bayes

In [14]:
# Pasamos nuestro np.array a un cupix array de enteros (Nos permite trabajar con CUDA).
y_train = cp.asarray(y_train, dtype=cp.int32)

# Instanciomos a nuestro clasificador Naive Bayes.
model = MultinomialNB()

# Mandamos a entrenar a nuestro clasificador.
tiempo_inicial = time.time()
model.fit(x_train, y_train)

print('Accuracy del clasificador: ', model.score(x_train, y_train))
tiempo_final = time.time()-tiempo_inicial
print('Tiempo aproximado consumido por el algoritmo (en nanosegundos): ', tiempo_final)


Accuracy del clasificador:  0.5538221597671509
Tiempo aproximado consumido por el algoritmo (en nanosegundos):  0.012101173400878906


# 7.Evaluacion del accuracy de nuestro clasificador NB.

In [10]:
# Utilizamos 20 oraciones al azar de nuestro conjunto de frases para Test.
x_test = x_test[0:20]

# Predecimos nuestro conjunto de test.
prediccion = model.predict(x_test)

# Transformamos nuestros resultados a np.array.
value_test = value_test.to_numpy()
expected_test = expected_test.to_numpy()

# Imprimos los resultados del entrenamiento.
for i in range(20):
    print('Frase: ', value_test[i], '\n', '--'*50, '\n', 'sentimiento predicho: ', prediccion[i], ', valor esperado: ', expected_test[i])
    print('__.__'*25)

Frase:  the attractive set use throughout most of the film be an eyeplease gem   
 ---------------------------------------------------------------------------------------------------- 
 sentimiento predicho:  1 , valor esperado:  1
__.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.__
Frase:  this be one of the bad film i have ever see   
 ---------------------------------------------------------------------------------------------------- 
 sentimiento predicho:  0 , valor esperado:  0
__.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.____.__
Frase:  the microphone also work well but accord to people i have call it applifie everything 
 ---------------------------------------------------------------------------------------------------- 
 sentimiento predicho:  0 , valor esperado:  1
__.____.____.____.____.____.____.____.____.____.____.____.____.___